# First steps with the pyam package

## Scope and feature overview

The **pyam** package provides a range of diagnostic tools and functions
for analyzing, visualizing and working with timeseries data following the format established by the *Integrated Assessment Modeling Consortium* ([IAMC](http://www.globalchange.umd.edu/iamc/)). The format has been used in several IPCC assessments and numerous model comparison exercises.

<img style="float: right; margin: 10px;" src="_static/IAMC_logo.jpg">

An illustrative example of this format template is shown below;
[read the docs](https://pyam-iamc.readthedocs.io/en/stable/data.html) for more information.


| **Model** | **Scenario** | **Region** | **Variable**   | **Unit** | **2005** | **2010** | **2015** |
|-----------|--------------|------------|----------------|----------|----------|----------|----------|
| MESSAGE   | CD-LINKS 400 | World      | Primary Energy | EJ/y     |    462.5 |    500.7 |      ... |

This notebook illustrates the basic functionality of the **pyam** package
and the ``IamDataFrame`` class:

0. Load timeseries data from a snapshot file and inspect the scenario ensemble.
1. Apply filters to the ensemble and display the timeseries data 
   as [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).
2. Visualize timeseries data using the plotting library based on the [matplotlib](https://matplotlib.org/) package.
3. Perform scenario diagnostic and validation checks.
5. Evaluating the model data and executing a range of diagnostic checks for identifying outliers.
6. Categorization of scenarios according to timeseries data values or checks on required variables.
7. Exporting data to `xlsx` using the IAMC template.


## Read the docs

A comprehensive documentation of the **pyam** package is available at [pyam-iamc.readthedocs.io](http://pyam-iamc.readthedocs.io).

## Tutorial data

The timeseries data used in this tutorial is a partial snapshot of the scenario ensemble
compiled for the IPCC's *Special Report on Global Warming of 1.5°C* ([SR15](http://ipcc.ch/sr15/)).
The complete scenario ensemble data is publicly available from the [IAMC 1.5°C Scenario Explorer and Data hosted by IIASA](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).  
*Please read the [License](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer/#/license) page of the IAMC 1.5°C Scenario Explorer before using the full scenario data for scientific analyis or other work.*

<img style="float: right; margin: 10px;" src="_static/cdlinks_logo.png">

### Scenarios in the tutorial data

The data snapshot used for this tutorial consists of selected data from the following sources:
 - an ensemble of scenarios from the *Horizon 2020* [CD-LINKS](https://www.cd-links.org) project  
 - the "Faster Transition Scenario" from the IEA's [World Energy Outlook 2017](https://www.oecd-ilibrary.org/energy/world-energy-outlook-2017_weo-2017-en),
 - the "1.0" scenario submitted by the GENeSYS-MOD team ([Löffler et al., 2017](https://doi.org/10.3390/en10101468))

Please refer to the [About](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer/#/about) page of the *IAMC 1.5°C Scenario Explorer* for references and additional information.

<div style="text-align: center; margin: 10px; width: 700px">
<div class="alert alert-block alert-warning">
    <i>The data used here is a partial snapshot of the IAMC 1.5°C Scenario Data!</i><br />
    <i>This tutorial is only intended as an illustration of the <b>pyam</b> package.</i>
</div></div>

### Citation of the scenario ensemble

> D. Huppmann, E. Kriegler, V. Krey, K. Riahi, J. Rogelj, K. Calvin, F. Humpenoeder, A. Popp, S. K. Rose, J. Weyant, et al.  
> *IAMC 1.5°C Scenario Explorer and Data hosted by IIASA* (release 2.0)  
> Integrated Assessment Modeling Consortium & International Institute for Applied Systems Analysis, 2019.  
> doi: [10.5281/zenodo.3363345](https://doi.org/10.5281/zenodo.3363345) | url: [data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer)

In [ ]:
import pyam
import matplotlib.pyplot as plt

## Load timeseries data from a snapshot file and inspect the scenario ensemble

We import the snapshot of the timeseries data from the file ``tutorial_data.csv``.

<div class="alert alert-block alert-info">
If you haven't cloned the <b>pyam</b> GitHub repository to your machine, you can download the file
from the folder <a href="https://github.com/IAMconsortium/pyam/tree/master/doc/source/tutorials">doc/source/tutorials</a>. <br />
Make sure to place the file in the same folder as this notebook.
</div>

In [ ]:
df = pyam.IamDataFrame(data='tutorial_data.csv')

As a first step, we show lists of all models, scenarios, regions, and the variables (including units) in the snapshot.

In [ ]:
df.models()

In [ ]:
df.scenarios()

In [ ]:
df.regions()

In [ ]:
df.variables(include_units=True)

## Apply filters to the ensemble and display the timeseries data

A selection of the timeseries data  of an ``IamDataFrame`` can be obtained by applying the [filter()](https://pyam-iamc.readthedocs.io/en/stable/api.html#pyam.IamDataFrame.filter) function,
which takes keyword-arguments of criteria.
The function returns a down-selected clone of the ``IamDataFrame``.

### Filtering by model names, scenarios and regions

The feature for filtering by **model, scenario or region** 
are implemented using exact string matching, where ``*`` can be used as a wildcard.

First, we want to display the list of all scenarios submitted by the **MESSAGE** modeling team.

> Applying the filter argument ``model='MESSAGE'`` will return an empty array  
> (because the **MESSAGE** model in the tutorial data is actually called **MESSAGEix-GLOBIOM 1.0**)

In [ ]:
df.filter(model='MESSAGE').scenarios()

> Filtering for ``model='MESSAGE*'`` will return all scenarios provided by the **MESSAGEix-GLOBIOM 1.0** model

In [ ]:
df.filter(model='MESSAGE*').scenarios()

### Inverting the selection

Using the keyword `keep=False` allows you to select the inverse of the filter arguments.

In [ ]:
df.filter(region='World').regions()

In [ ]:
df.filter(region='World', keep=False).regions()

### Filtering by variables and levels

Filtering for **variable** strings works in an identical way as above,
with ``*`` available as a wildcard.

> Filtering for ``Primary Energy`` will return only exactly those data

> Filtering for ``Primary Energy|*`` will return all sub-categories of 
> primary energy (and only the sub-categories)

In additon, variables can be filtered by their **level**,
i.e., the "depth" of the variable in a hierarchical reading of the string separated by `|` (*pipe*, not L or i).
That is, the variable ``Primary Energy`` has level 0, while ``Primary Energy|Fossil`` has level 1.

Filtering by both **variables** and **level** will search for the hierarchical depth 
_following the variable string_ so filter arguments ``variable='Primary Energy*'`` and ``level=1``
will return all variables immediately below ``Primary Energy``.
Filtering by **level** only will return all variables at that depth.

In [ ]:
df.filter(variable='Primary Energy*', level=1).variables()

The next cell illustrates another use case of the **level** filter argument - filtering by `1-` (as string) instead of `1` (as integer) will return all timeseries data for variables *up to* the specified depth.

In [ ]:
df.filter(variable='Primary Energy*', level='1-').variables()

The last cell shows how to filter only by **level** without providing a **variable** argument.
The example returns all variables that are at the second hierarchical level (i.e., not ``Primary Energy``).

In [ ]:
df.filter(level=1).variables()

### Filtering by year

Filtering for **years** can be done by one integer value, a list of integers, or the Python class [range](https://docs.python.org/3/library/stdtypes.html#ranges).

**Note**: the last year of a range is not included, so ``range(2010, 2015)``
is interpreted as ``[2010, 2011, 2012, 2013, 2014]``.

### Displaying timeseries data

As a next step, we want to view a selection of the timeseries data.
The [timeseries()](https://pyam-iamc.readthedocs.io/en/stable/api.html#pyam.IamDataFrame.timeseries) function returns the data as a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) in the standard IAMC format, i.e., _wide format_ where years are shown as columns.

In [ ]:
display_df = df.filter(model='MESSAGE*', variable='Primary Energy', region='World')
display_df.timeseries()

### Parallels to the *pandas* data analysis toolkit

When developing **pyam**, we followed the syntax of the Python package **pandas** ([read the docs](https://pandas.pydata.org)) closely where possible. In many cases, you can use of similar functions directly on the ``IamDataFrame``.

In the next cell, we illustrate this parallel behaviour. The function [pyam.IamDataFrame.head()](https://pyam-iamc.readthedocs.io/en/stable/api.html#pyam.IamDataFrame.head) is similar to [pandas.DataFrame.head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html): 
it returns the first n rows of the ``data`` table in *long format* (i.e., columns are in year/value format).

In [ ]:
display_df.head()

### Getting help

When in doubt, you can look at the help for any function by appending a ``?``.

In [ ]:
df.filter?

## Visualize timeseries data using the plotting library

This section provides an illustrative example of the plotting features of the **pyam** package. Please look at the [plotting gallery](https://pyam-iamc.readthedocs.io/en/stable/examples/index.html) for more examples.

In the next cell, we show a simple line plot of global CO2 emissions. The colours are assigned randomly by default, and **pyam** deactivates the legend if there are too many lines.

In [ ]:
df.filter(variable='Emissions|CO2', region='World').line_plot()

For displaying data in a different format, the class ``IamDataFrame`` has a wrapper of the ``pd.DataFrame.pivot_table()`` function. It allows to flexibly specify the columns and rows.
The function automatically aggregates by summation or counting (specified by the parameter `aggfunc`) 
over all timeseries data identifiers ('model', 'scenario', 'variable', 'region', 'unit', 'year')
which are not used as `index` or `columns`.

In the example below, the filter of the timeseries data is set for all subcategories of 'Primary Energy', 
which are then summed up in the displayed table.

In [ ]:
(df
 .filter(variable='Primary Energy', region='World')
 .pivot_table(index=['year'], columns=['scenario'], values='value', aggfunc='sum')
)

If you are familiar with the `python` package `pandas`, you can use many of usual functions directly on the ``IamDataFrame``. The function ``head()``, for example, will show the first `n` rows of the data in long form 
(columns are in year/value format).

In [ ]:
df.head()

## Visualization of timeseries

This section provides one illustrative example of the plotting features of the ``pyam`` package. Please see the [plotting gallery](https://pyam-iamc.readthedocs.io/en/latest/examples/index.html) for more examples on plotting with `pyam`.

In [ ]:
df.filter(variable='Emissions|CO2', region='World').line_plot(legend=False)

## Validation and diagnostic assessment of timeseries data

When analyzing scenario results, it is often useful to check whether certain timeseries exist or the values are within a specific range. For example, it may make sense to ensure that reported data for historical periods are close to established reference data or that near-term developments are reasonable.

The following section provides three illustrations:
1. Check whether a timeseries `'Primary Energy'` exists in each scenario (in at least one year).
2. Check for every scenario whether the value for `'Primary Energy'` at the global level exceeds 515 EJ/y 
   in the reference year 2010
   (the value must satisfy an upper bound of 515 EJ/y in this notation).
3. Check for every scenario from the `AMPERE` project
   whether the value for `'Primary Energy|Coal'` exceeds 400 EJ/y in mid-century.

The `validate()` function performs the checks on models and scenarios.

The ``criteria`` argument can specify a valid range by an upper and lower bound (``up``, ``lo``) for a variable and a subset of years to which the validation is applied - all scenarios with a value in at least one year outside that range are considered to *not satisfy* the validation.

By setting the argument ``exclude=True``, all scenarios failing the validation will be categorized as ``exclude`` in the metadata. This allows to remove these scenarios from subsequent analysis or figures.

In [ ]:
df.require_variable(variable='Primary Energy')

In [ ]:
df.validate(criteria={'Primary Energy': {'up': 515, 'year': 2010}})

In [ ]:
pyam.validate(df.filter(region='World', scenario='AMPERE*'), 
              criteria={'Primary Energy|Coal': {'up': 400, 'year': 2050}}
)

## Categorization of scenarios by timeseries characteristics

It is often useful to apply categorization to classes of scenarios according to specific characteristics of the timeseries data. In the following example, we use the temperature change assessment by MAGICC 6 to group scenarios by the median global warming by the end of the century (year 2100).

We proceed in the following steps:

0. Plot the timeseries data of the variable that we want to use. 
   This provides some insights on useful thresholds for the categorization.
0. Use the function ``categorize()`` to apply a categorization (and colour code for later use) 
   to all scenarios that satisfy a number of specific criteria.
0. Use the categorization of scenarios for analysis of other timeseries data.

In [ ]:
v = 'Temperature|Global Mean|MAGICC6|MED'
df.filter(region='World', variable=v).line_plot(legend=False)

We now use the categorization feature of the ``pyam`` package to group scenarios by temperature outcome by the end of the century.

The first cell sets the ``'Temperature'`` categorization to the default `"uncategorized"`.
This is not necessary per se (setting a meta column via the categorization will mark all non-assigned rows as `"uncategorized"` (if the value is a string) or `np.nan`.
Still, having this cell may be helpful in this tutorial if you are going back and forth between cells to reset the assignment.

The function `categorize()` takes `color` and similar arguments, which can then be used by the plotting library.

In [ ]:
df.set_meta(meta='uncategorized', name='Temperature')

In [ ]:
df.categorize(
    'Temperature', 'Below 1.6C',
    criteria={v: {'up': 1.6, 'year': 2100}},
    color='cornflowerblue'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.0C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 2.0, 'lo': 1.6, 'year': 2100}},
    color='forestgreen'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.5C',
    criteria={v: {'up': 2.5, 'lo': 2.0, 'year': 2100}},
    color='gold'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 3.5C',
     criteria={v: {'up': 3.5, 'lo': 2.5, 'year': 2100}},
     color='firebrick'
)

In [ ]:
df.categorize(
    'Temperature', 'Above 3.5C',
    criteria={v: {'lo': 3.5, 'year': 2100}},
    color='magenta'
)

Two models included in the snapshot have not been assessed by MAGICC6 regarding their long-term climate and warming impact. Therefore, the timeseries ``'Temperature|Global Mean|MAGICC6|MED'`` does not exist, and they have not been categorized.

In [ ]:
df.require_variable(variable=v, exclude_on_fail=False)

Now, we again display the median global temperature increase for all scenarios, but we use the colouring by category to illustrate the common charateristics across scenarios.

In [ ]:
df.filter(variable=v).line_plot(color='Temperature', legend=True)

As a last step, we display the aggregate CO2 emissions by category. This allows to highlight alternative pathways within the same category. 

In this step, we also export this figure as a png using the option ``savefig``. The figure will be saved in the tutorials folder.

In [ ]:
fig, ax = plt.subplots()
(df
 .filter(variable='Emissions|CO2', region='World')
 .line_plot(ax=ax, color='Temperature', legend=True)
)
fig.savefig('co2_emissions.png')

## Using metadata indicators for further scenario characterization & diagnostics

In the example above, we classified scenarios by their end-of-century warming. However, for some research questions, it may be of interest to know the maximum temperature over the course of the century (a.k.a. peak warming).

In this section, we illustrate two ways to add quantitative metadata indicators.
First, we add indicators derived directly from the data in the `IamDataFrame`.

In [ ]:
eoc = 'End-of-century-temperature'
df.set_meta_from_data(name='End-of-century-temperature', variable=v, year=2100)

If the filter arguments passed to `set_meta_from_data()` do not yield a unique value,
we can pass a `method` to aggregate.

In [ ]:
peak = 'Peak-temperature'
df.set_meta_from_data(name='Peak-temperature', variable=v, method=np.max)

The second method to define metadata indicators can take any `pandas.Series` with an index including `model` and `scenario`.

In the example, we can now easily derive the "overshoot", i.e., the reduction in global temperature after the peak,
by computing the difference between the two metadata indicators.

In [ ]:
overshoot = df.meta[peak] - df.meta[eoc]

In [ ]:
df.set_meta(name='Overshoot', meta=overshoot)

As a last step of the illustration, we show the first rows of the `meta` dataframe
downselected to the scenarios in the Temperature-category `Below 2.0C`.
This table now includes the three new metadata indicators.

In [ ]:
df.filter(Temperature='Below 2.0C').meta.head()

## Exporting timeseries data for further analysis

The `IamDataFrame` can be exported to `xlsx` and `csv` in the IAMC (wide) format. When writing to `xlsx`, both the timeseries data and the `meta` dataframe of categorization and quantitative indicators will be written to the file,
to two sheets named `data` and `meta` respectively.

This feature is based on [pd.DataFrame.to_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html) and [pd.DataFrame.to_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html). It can use any keyword arguments of that function.

In [ ]:
df.to_excel('tutorial_export.xlsx')